# Analysis for WM_NoiseV11 in Python instead of R because pandas > tidyverse
### HKular 11.7.23

# Import libraries and load data, some data wrangling

In [11]:
os.getlogin()

'hollykular'

In [9]:
## Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

# Set the working directory
import os
os.chdir('/Users/' os.getlogin() '/Documents/Github/noisefx/Starburst_V11/')

# Load data
V11 = pd.read_csv('WM_noiseV11.csv') # good subjects
V11b = pd.read_csv('WM_noiseV11_bad.csv') # bad subjects to compare

# Make numerical values that are factors actually factors
factor_mapping = {
    'distractor': {1: 'none', 2: 'ignore', 3: 'attend'},
    'kappa': {100: 'high', 5000: 'low'},
    'change': {10: 'ccw', -10: 'cw'},
    'dist': {114: 'ccw', 112: 'ccw', 117: 'cw', 115: 'cw'}
}

V11.replace(factor_mapping, inplace=True)
V11b.replace(factor_mapping, inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hkular/Documents/Github/noisefx/Starburst_V11'

# Individual subject quality check

In [8]:
## If they withdraw from the experiment before completing all trials (n = 576)
ntrials = V11['subject'].value_counts()
# V11 = V11[~V11['subject'].isin(insert attrition here)]

# Responded to at least 90% of the distractors in the attend-distractor condition
attendrespcheck = V11[V11['distractor'] == 'attend'].groupby('subject')['dist'].count()
attendrespcheck = attendrespcheck[attendrespcheck.isna()]
for subject, count in attendrespcheck.items():
    if count > 0.1 * ntrials[subject] / 3:
        print(f"exclude subject {subject} no response to attend")

# Responded to more than 10% of the distractors in the ignore-distractor condition
ignorerespcheck = V11[V11['distractor'] == 'ignore'].groupby('subject')['dist'].count()
ignorerespcheck = ignorerespcheck[ignorerespcheck.isna()]
for subject, count in ignorerespcheck.items():
    if count < 0.9 * ntrials[subject] / 3:
        print(f"exclude subject {subject} response to ignore")

# Responded to at least 90% of the targets
nonrespcheck = V11.groupby('subject')['resp'].count()
nonrespcheck = nonrespcheck[nonrespcheck.isna()]
for subject, count in nonrespcheck.items():
    if count > 0.1 * ntrials[subject]:
        print(f"exclude subject {subject} no response to target")

# Subjects will be removed if there is evidence for responses clustered around the orientation of the distractor
# Instead of around the memory target, a cluster of responses around the distractor orientation suggests non-compliance with task instructions.
taskcheck = V11.dropna(subset=['distractorori', 'resp'])

# The average measured performance error is greater than 49 degrees
perfcheck = taskcheck.groupby('subject').agg(
    circ_mean=('acc', lambda x: circmean(x * np.pi / 180)),
    circ_std=('acc', lambda x: circstd(x * np.pi / 180))
)
perfcheck = perfcheck[perfcheck['circ_std'] * 180 / np.pi > 49]
for subject in perfcheck.index:
    print(f"exclude subject {subject} large behavior error")

# Clean up
del attendrespcheck, ignorerespcheck, ntrials, taskcheck, perfcheck, nonrespcheck

# Create scatter plot (You might need to customize this part)
plt.scatter(V11['distractorori'], V11['resp'])
plt.xlabel("Distractor (degrees)")
plt.ylabel("Response (degrees)")
plt.show()

NameError: name 'V11' is not defined